In [350]:
import numpy as np

Матрица B для расширенного кода Голея

In [351]:
B = np.array([[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
              [1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1],
              [0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1],
              [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
              [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
              [1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
              [0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
              [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
              [0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1],
              [1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1],
              [0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
              [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])

Нахождение порождающей и проверочной матриц расширенного кода Голея

In [352]:

def get_generative_and_verification_galley_matrixes():
    G = np.hstack((np.eye(12, 12, dtype=int), B))
    H = np.vstack((np.eye(12, 12, dtype=int),B))
    return G, H

Обнаружение и исправление n-кратной ошибки с помощью расширенного кода Голея

In [353]:
def fix_error_with_galley_code(G, H, n):
    U = np.array([0] * len(G))
    w = np.mod(np.dot(U, G), 2)
    e = w.copy()

    # вводим n-кратную ошибку
    for i in range(n):
        e[i] = (e[i] + 1) % 2
    s = np.mod(np.dot(e, H), 2)

    corr = None
    if (sum(s) <= 3):
        corr = np.hstack((s, np.zeros(len(s), dtype=int)))
    for i in range(len(B)):
        if(sum(s ^ B[i]) <= 2):
            err_el = np.zeros(len(s))
            err_el[i] = 1
            corr=np.hstack(s ^ B[i], err_el)
    if corr is None:
        sec_s = np.mod(np.dot(s, B), 2)
        if (sum(sec_s) <= 3):
            corr = np.hstack(( np.zeros(len(s), dtype=int),sec_s))
        for j in range(len(B)):
            if (sum(sec_s ^ B[j]) <= 2):
                err_el = np.zeros(len(s))
                err_el[j] = 1
                corr = np.hstack(err_el,sec_s ^ B[j])
    if corr is not None:
        corr=corr ^ e
    return "Ошибка обнаружена, исправлена" if np.array_equal(corr,w) else "Ошибка обнаружена, не исправлена"


Генерация кода Рида-Маллера

In [354]:
def generate_rid_maller_code(r, m):
    if r == 0:
        return np.ones((1, 2 ** m), dtype=int)
    if r == m:
        G = generate_rid_maller_code(m - 1, m)
        return np.vstack([G, np.eye(1, 2 ** m, -1, dtype=int)])
    G_rm1 = generate_rid_maller_code(r, m - 1)
    G_r1_m1 = generate_rid_maller_code(r - 1, m - 1)
    return np.vstack([np.hstack([G_rm1, G_rm1]),
                      np.hstack([np.zeros_like(G_r1_m1, dtype=int), G_r1_m1])])

Создание проверочной матрицы кода Рида-Маллера

In [355]:
def verification_RM_matrix(i, m):
    H = np.array([[1, 1], [1, -1]], dtype=int)
    return np.kron(np.kron(np.eye(2**(m-i), dtype=int), H), np.eye(2**(i-1), dtype=int))


Замена 0 на -1, необходимая для кода Рида-Маллера

In [356]:
def replace_on_minus_one(w):
    return np.where(w == 0, -1, 1)

Исправление ошибок с помощью Рида-Маллера

In [ ]:
def fix_RM_errors(G, m, n):
    U = np.zeros(G.shape[0], dtype=int)
    w = np.mod(U @ G, 2)
    e1 = w.copy()
    e1[:n] = (e1[:n] + 1) % 2 

    wm = replace_on_minus_one(e1) @ verification_RM_matrix(1, m)
    for j in range(2, m + 1):
        wm = wm @ verification_RM_matrix(j, m)

    lead = np.argmax(np.abs(wm))
    v = np.array(list(map(int, bin(lead)[2:].zfill(m)[::-1])))

    u = np.append([wm[lead] > 0], v).astype(int)
    return "Ошибка обнаружена, исправлена" if np.array_equal(u, U) else "Ошибка обнаружена, не исправлена"


Работа с кодом Голея

In [358]:
g_galley, h_galley = get_generative_and_verification_galley_matrixes()
print("Матрица g_galley\n",g_galley)
print("Матрица h_galley\n",h_galley)
print("Исправление ошибок для однократной ошибки: ", fix_error_with_galley_code(g_galley, h_galley, 1))
print("Исправление ошибок для двухкратной ошибки: ", fix_error_with_galley_code(g_galley, h_galley, 2))
print("Исправление ошибок для трехкратной ошибки: ", fix_error_with_galley_code(g_galley, h_galley, 3))
print("Исправление ошибок для четырехкратной ошибки: ", fix_error_with_galley_code(g_galley, h_galley, 4), '\n')

Матрица g_galley
 [[1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 0 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 1 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0]]
Матрица h_galley
 [[1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 0 1 1 1 0 0 0 1 0 1]


Работа с кодом Рида-Маллера

In [359]:
r, m = 1, 3
RM_G_matr=generate_rid_maller_code(r,m)
print("Матрица G кода Рида-Маллера r = 1 m = 3", RM_G_matr)
print("Исправление ошибок для однократной ошибки", fix_RM_errors(RM_G_matr, m, 1))
print("Исправление ошибок для двухкратной ошибки", fix_RM_errors(RM_G_matr, m, 2))

r, m = 1, 4
RM_G_matr=generate_rid_maller_code(r,m)
print("Матрица G кода Рида-Маллера r = 1 m = 4",RM_G_matr)
print("Исправление ошибок с помощью кода Рида-Маллера при r = 1 m = 4")
print("Исправление ошибок для однократной ошибки", fix_RM_errors(RM_G_matr, m, 1))
print("Исправление ошибок для двухкратной ошибки", fix_RM_errors(RM_G_matr, m, 2))
print("Исправление ошибок для трёхкратной ошибки", fix_RM_errors(RM_G_matr, m, 3))
print("Исправление ошибок для четырёхкратной ошибки", fix_RM_errors(RM_G_matr, m, 4))

Матрица G кода Рида-Маллера r = 1 m = 3 [[1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 1 1]
 [0 0 0 0 1 1 1 1]]
Исправление ошибок для однократной ошибки Ошибка обнаружена, исправлена
Исправление ошибок для двухкратной ошибки Ошибка обнаружена, исправлена
Матрица G кода Рида-Маллера r = 1 m = 4 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
 [0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]]
Исправление ошибок с помощью кода Рида-Маллера при r = 1 m = 4
Исправление ошибок для однократной ошибки Ошибка обнаружена, исправлена
Исправление ошибок для двухкратной ошибки Ошибка обнаружена, исправлена
Исправление ошибок для трёхкратной ошибки Ошибка обнаружена, исправлена
Исправление ошибок для четырёхкратной ошибки Ошибка обнаружена, исправлена
